## 3. Tratamentos Iniciais dos Dataframes - Camada Silver

Nesse notebook, realizamos o primeiro processo de tratamento das tabelas `customer_profiles_bronze`, `terminal_profiles_bronze` e `transaction_bronze`, Localizada na camada Bronze do nosso Lakehouse. Os dados foram inicialmente carregados em seu formato bruto e passaram pelas seguintes etapas: 

1. Verificação do Schema Original
2. Ajuste dos Tipos de Dados
3. Validação de Dados Nulos
4. Gravação na Camada Silver

### imports

In [0]:
from pyspark.sql.functions import col, from_json, to_timestamp
from pyspark.sql.types import ArrayType, IntegerType


### 3.1. Tratamento na Tabela Customer

In [0]:
customer = spark.sql("""
SELECT * FROM workspace.fraud_detection.customer_profiles_bronze
""")


In [0]:
customer.show(3)

Utilizamos o método printSchema() para inspecionar os tipos de dados originais das colunas. Identificamos que todos os campos estavam como string, inclusive colunas numéricas e arrays, devido à forma como os dados foram ingeridos.

In [0]:
customer.printSchema()


Realizamos a conversão dos tipos de dados de cada coluna para seus formatos adequados (ex: double, int, array<int>), garantindo que as colunas possam ser utilizadas corretamente em análises estatísticas e modelagens futuras.

In [0]:
customer = customer \
    .withColumn("x_customer_id", col("x_customer_id").cast("double")) \
    .withColumn("y_customer_id", col("y_customer_id").cast("double")) \
    .withColumn("mean_amount", col("mean_amount").cast("double")) \
    .withColumn("std_amount", col("std_amount").cast("double")) \
    .withColumn("mean_nb_tx_per_day", col("mean_nb_tx_per_day").cast("double")) \
    .withColumn("nb_terminals", col("nb_terminals").cast("int"))


A coluna available_terminals, originalmente armazenada como uma string no formato de lista ("[1, 2, 3]"), foi convertida para um tipo array<int> utilizando a função from_json.

In [0]:
schema_array = ArrayType(IntegerType())

customer = customer.withColumn(
    "available_terminals",
    from_json(col("available_terminals"), schema_array)
)


In [0]:
customer.printSchema()


Utilizamos uma query SQL para verificar a presença de valores nulos em todas as colunas. Como os dados foram simulados, nenhuma ausência foi encontrada.

In [0]:
%sql
SELECT
  COUNT(*) AS total_linhas,
  SUM(CASE WHEN CUSTOMER_ID IS NULL THEN 1 ELSE 0 END) AS nulos_CUSTOMER_ID,
  SUM(CASE WHEN x_customer_id IS NULL THEN 1 ELSE 0 END) AS nulos_x_customer_id,
  SUM(CASE WHEN y_customer_id IS NULL THEN 1 ELSE 0 END) AS nulos_y_customer_id,
  SUM(CASE WHEN mean_amount IS NULL THEN 1 ELSE 0 END) AS nulos_mean_amount,
  SUM(CASE WHEN std_amount IS NULL THEN 1 ELSE 0 END) AS nulos_std_amount,
  SUM(CASE WHEN mean_nb_tx_per_day IS NULL THEN 1 ELSE 0 END) AS nulos_mean_nb_tx_per_day,
  SUM(CASE WHEN available_terminals IS NULL THEN 1 ELSE 0 END) AS nulos_available_terminals,
  SUM(CASE WHEN nb_terminals IS NULL THEN 1 ELSE 0 END) AS nulos_nb_terminals
FROM workspace.fraud_detection.customer_profiles_bronze;


Após os ajustes, os dados foram persistidos na camada Silver como a tabela `customer_profiles_silver`, agora com estrutura limpa, padronizada e pronta para enriquecimentos, validações de consistência e análises exploratórias.

In [0]:
customer.write.format("delta").mode("overwrite").saveAsTable("fraud_detection.customer_profiles_silver")

### 3.2. Tratamento na Tabela Terminal

In [0]:
terminal = spark.sql("""
SELECT * FROM workspace.fraud_detection.terminal_profiles_bronze
""")


In [0]:
terminal.show(3)

Utilizamos o método printSchema() para inspecionar os tipos de dados originais das colunas. Identificamos que todos os campos estavam como string, devido à forma como os dados foram ingeridos.


In [0]:
terminal.printSchema()


Realizamos a conversão dos tipos de dados de cada coluna para seus formatos adequados (ex: double, int), garantindo que as colunas possam ser utilizadas corretamente em análises estatísticas e modelagens futuras.


In [0]:
terminal = terminal \
    .withColumn("x_terminal_id", col("x_terminal_id").cast("double")) \
    .withColumn("y_terminal_id", col("y_terminal_id").cast("double"))


In [0]:
terminal.printSchema()

Utilizamos uma query SQL para verificar a presença de valores nulos em todas as colunas. Como os dados foram simulados, nenhuma ausência foi encontrada.


In [0]:
%sql
SELECT
  COUNT(*) AS total_linhas,
  SUM(CASE WHEN TERMINAL_ID IS NULL THEN 1 ELSE 0 END) AS nulos_TERMINAL_ID,
  SUM(CASE WHEN x_terminal_id IS NULL THEN 1 ELSE 0 END) AS nulos_x_terminal_id,
  SUM(CASE WHEN y_terminal_id IS NULL THEN 1 ELSE 0 END) AS nulos_y_terminal_id
FROM workspace.fraud_detection.terminal_profiles_bronze;


Após os ajustes, os dados foram persistidos na camada Silver como a tabela `terminal_profiles_silver`, agora com estrutura limpa, padronizada e pronta para enriquecimentos, validações de consistência e análises exploratórias.


In [0]:
terminal.write.format("delta").mode("overwrite").saveAsTable("fraud_detection.terminal_profiles_silver")

### 3.3. Tratamento na Tabela Transaction

In [0]:
transactions = spark.sql("""
SELECT * FROM workspace.fraud_detection.transaction_bronze
""")


In [0]:
transactions.show(3)

Utilizamos o método printSchema() para inspecionar os tipos de dados originais das colunas. Identificamos que todos os campos estavam como string, devido à forma como os dados foram ingeridos.

In [0]:
transactions.printSchema()

Realizamos a conversão dos tipos de dados de cada coluna para seus formatos adequados (ex: double, int, timestamp), garantindo que as colunas possam ser utilizadas corretamente em análises estatísticas e modelagens futuras.

In [0]:
transactions = transactions \
    .withColumn("TX_DATETIME", to_timestamp(col("TX_DATETIME"), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("TX_AMOUNT", col("TX_AMOUNT").cast("double")) \
    .withColumn("TX_TIME_SECONDS", col("TX_TIME_SECONDS").cast("int")) \
    .withColumn("TX_TIME_DAYS", col("TX_TIME_DAYS").cast("int")) \
    .withColumn("TX_FRAUD", col("TX_FRAUD").cast("int")) \
    .withColumn("TX_FRAUD_SCENARIO", col("TX_FRAUD_SCENARIO").cast("int"))


In [0]:
transactions.printSchema()

In [0]:
transactions.show(3)

Utilizamos uma query SQL para verificar a presença de valores nulos em todas as colunas. Como os dados foram simulados, nenhuma ausência foi encontrada.


In [0]:
%sql

SELECT 
  count(*) as total_linhas,
  sum(case when TRANSACTION_ID is null then 1 else 0 end) as nulos_TRANSACTION_ID,
  sum(case when TX_DATETIME is null then 1 else 0 end) as nulos_TX_DATETIME,
  sum(case when CUSTOMER_ID is null then 1 else 0 end) as nulos_CUSTOMER_ID,
  sum(case when TERMINAL_ID is null then 1 else 0 end) as nulos_TERMINAL_ID,
  sum(case when TX_AMOUNT is null then 1 else 0 end) as nulos_TX_AMOUNT,
  sum(case when TX_TIME_SECONDS is null then 1 else 0 end) as nulos_TX_TIME_SECONDS,
  sum(case when TX_TIME_DAYS is null then 1 else 0 end) as nulos_TX_TIME_DAYS,
  sum(case when TX_FRAUD is null then 1 else 0 end) as nulos_TX_FRAUD,
  sum(case when TX_FRAUD_SCENARIO is null then 1 else 0 end) as nulos_TX_FRAUD_SCENARIO
FROM workspace.fraud_detection.transaction_bronze;


Após os ajustes, os dados foram persistidos na camada Silver como a tabela `transaction_silver`, agora com estrutura limpa, padronizada e pronta para enriquecimentos, validações de consistência e análises exploratórias.


In [0]:
transactions.write.format("delta").mode("overwrite").saveAsTable("fraud_detection.transaction_silver")